# S3VT Landsat and Sentinel 2 validation of hotspots - working

## Description
This notebook demonstrates how to:
 
From a candidate latitude longitude and solar_day:
* determine if intersecting Landsat or Sentinel 2 ARD exists
* apply the platform specific tests to determine if hotspots were detected in the vicinity 5km of hotspot
* return number of pixel identified as hotspots
* save a boolean file labelled with solar date of acquisition
* as a secondary test perform a Normalized Burnt Ratio and return as a binary with solar date of acquisition
    * find canidate dates within a time range of source hotspot
        * find closest before date within tolerance (dNBR A)
        * find closest after date within tolerance (dNBR B)
        * candidate closest to source hotspot will be used for hotspot matching i.e. high resolution hotspot

Assumptions:
* reflectance values are scaled by 10000 i.e. 100% reflectance = 10000
 

### Load packages

In [1]:
%matplotlib inline
from pathlib import Path
import datacube
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
import sys
import xarray as xr
import geopandas as gpd

sys.path.append("Scripts")
from dea_datahandling import load_ard
from dea_plotting import rgb
from dea_bandindices import calculate_indices
from shapely import wkt
from geopy.distance import distance

import rioxarray

### Connect to the datacube

In [2]:
dc = datacube.Datacube(app='validating_hotspots')

### Be ignorant of the sensor

In [3]:
# configure sensor bands - #TODO implement sensor ignorance code here
sensor_ignorance = {'msi':{'0.433-0.453': 'nbar_coastal_aerosol',
                           '0.450-0.515': 'nbar_blue',
                           '0.525-0.600': 'nbar_green',
                           '0.630-0.680': 'nbar_red',
                           '0.845-0.885': 'nbar_nir_1',
                           '1.560-1.660': 'nbar_swir_2',
                           '2.100-2.300': 'nbar_swir_3',
                           'fmask': 'fmask'},
                   'oli': {'0.433-0.453': 'nbart_band01',
                           '0.450-0.515': 'nbart_band02',
                           '0.525-0.600': 'nbart_band03',
                           '0.630-0.680': 'nbart_band04',
                           '0.845-0.885': 'nbart_band05',
                           '1.560-1.660': 'nbart_band06',
                           '2.100-2.300': 'nbart_band07',
                           'fmask': 'fmask'}}

In [4]:
df = pd.read_csv('nearest_points.SENTINEL_3A_SLSTR_ESA.csv')
geometry = df.geometry.apply(wkt.loads)
crs = 'epsg:4326'
nearest_points = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
nearest_points['datetime'] = pd.to_datetime(nearest_points['datetime'])
nearest_points['solar_day'] = pd.to_datetime(nearest_points['solar_day'])
nearest_points['2_datetime'] = pd.to_datetime(nearest_points['2_datetime'])
nearest_points['dist_m'] = nearest_points.apply(lambda row: distance((row.latitude, row.longitude),(row['2_latitude'], row['2_longitude'])).meters, axis = 1)
nearest_points['timedelta'] = (abs(nearest_points['datetime'] - nearest_points['2_datetime']))
nearest_points['count'] = 1
nearest_points = nearest_points[(nearest_points['dist_m'] < 5000)]

In [95]:
nearest_points["s2msi_rdnbr_gt_1200"] = ""
nearest_points["s2msi_pre_burn_time"] = ""
nearest_points["s2msi_post_burn_time"] = ""
nearest_points["s2msi_pre_burn_timedelta"] = ""
nearest_points["s2msi_post_burn_timedelta"] = ""
nearest_points["s2msi_pre_percent"] = ""
nearest_points["s2msi_post_percent"] = ""
nearest_points["lsoli_rdnbr_gt_1200"] = ""
nearest_points["lsoli_pre_burn_time"] = ""
nearest_points["lsoli_post_burn_time"] = ""
nearest_points["lsoli_pre_burn_timedelta"] = ""
nearest_points["lsoli_post_burn_timedelta"] = ""
nearest_points["lsoli_pre_percent"] = ""
nearest_points["lsoli_post_percent"] = ""
nearest_points["lsoli_hotspot_percent"] = ""

# Functions

In [6]:
# Buffer candidate hotspot with 5 kilometre radius (or .05 degrees will do)
def buffer_hotspot(lon, lat):
    ul_lon = lon - 0.05
    lr_lon = lon + 0.05
    ul_lat = lat + 0.05
    lr_lat = lat - 0.05
    return ((ul_lon, lr_lon), (ul_lat, lr_lat))

In [7]:
#def merge(list1, list2, list3): 
      
#    merged_list = [(list1[i], list2[i], list3[i]) for i in range(0, len(list1))] 
#    return merged_list
#nearest_points.datetime[0].date()
#datelist = []
#for i in nearest_points.datetime:
#    datelist.append(i)#.date())#.strftime("%Y-%m-%d"))
#    
#candidate_list = list(set(merge(nearest_points.longitude.to_list(),
#          nearest_points.latitude.to_list(),
#          datelist)))

In [8]:
def buffer_date(firetime, days):
    prefire_date = (firetime - np.timedelta64(days, "D")).astype(str)
    postfire_date = (firetime + np.timedelta64(days, "D")).astype(str)
    return(prefire_date, postfire_date)

In [9]:
def get_measurement_list(product):
    measurement_list = []
    for i in dc.list_products().name:
        for j in dc.list_measurements().query('product == @i').name:
            if i == product:
                measurement_list.append([i, '--',j])
    return(measurement_list)

In [10]:
# potentially unambiguous active fire pixels
def get_candidates(ds):
    test1 = (((ds[sensor_ignorance[sensor]['2.100-2.300']] / ds[sensor_ignorance[sensor]['0.845-0.885']]) > 2.5) *
             ((ds[sensor_ignorance[sensor]['2.100-2.300']]  - ds[sensor_ignorance[sensor]['0.845-0.885']]) > 3000) *
             (ds[sensor_ignorance[sensor]['2.100-2.300']] > 5000))
    # Unambiguous fire pixels
    test2 = (((ds[sensor_ignorance[sensor]['1.560-1.660']] > 8000) *
              (ds[sensor_ignorance[sensor]['0.433-0.453']] < 2000)) *
             ((ds[sensor_ignorance[sensor]['0.845-0.885']] > 4000) +
              (ds[sensor_ignorance[sensor]['2.100-2.300']] < 1000)).clip(min=0, max=1))
    # other candidate fire pixels
    test3 = (((ds[sensor_ignorance[sensor]['2.100-2.300']]/ds[sensor_ignorance[sensor]['0.845-0.885']]) > 1.8)*
             (ds[sensor_ignorance[sensor]['2.100-2.300']]-ds[sensor_ignorance[sensor]['0.845-0.885']]  > 1700))
    unambiguous = (test1 + test2 + test3).clip(min=0, max=1)
    return(unambiguous)

In [11]:
get_measurement_list('ga_s2a_ard_nbar_granule')

[['ga_s2a_ard_nbar_granule', '--', 'azimuthal_exiting'],
 ['ga_s2a_ard_nbar_granule', '--', 'azimuthal_incident'],
 ['ga_s2a_ard_nbar_granule', '--', 'exiting'],
 ['ga_s2a_ard_nbar_granule', '--', 'incident'],
 ['ga_s2a_ard_nbar_granule', '--', 'relative_azimuth'],
 ['ga_s2a_ard_nbar_granule', '--', 'relative_slope'],
 ['ga_s2a_ard_nbar_granule', '--', 'satellite_azimuth'],
 ['ga_s2a_ard_nbar_granule', '--', 'satellite_view'],
 ['ga_s2a_ard_nbar_granule', '--', 'solar_azimuth'],
 ['ga_s2a_ard_nbar_granule', '--', 'solar_zenith'],
 ['ga_s2a_ard_nbar_granule', '--', 'terrain_shadow'],
 ['ga_s2a_ard_nbar_granule', '--', 'fmask'],
 ['ga_s2a_ard_nbar_granule', '--', 'nbar_contiguity'],
 ['ga_s2a_ard_nbar_granule', '--', 'nbar_coastal_aerosol'],
 ['ga_s2a_ard_nbar_granule', '--', 'nbar_blue'],
 ['ga_s2a_ard_nbar_granule', '--', 'nbar_green'],
 ['ga_s2a_ard_nbar_granule', '--', 'nbar_red'],
 ['ga_s2a_ard_nbar_granule', '--', 'nbar_red_edge_1'],
 ['ga_s2a_ard_nbar_granule', '--', 'nbar_red_edg

In [12]:
# potentially unambiguous active fire pixels
def get_candidates(ds):
    test1 = (((ds[sensor_ignorance[sensor]['2.100-2.300']] / ds[sensor_ignorance[sensor]['0.845-0.885']]) > 2.5) *
             ((ds[sensor_ignorance[sensor]['2.100-2.300']]  - ds[sensor_ignorance[sensor]['0.845-0.885']]) > 3000) *
             (ds[sensor_ignorance[sensor]['2.100-2.300']] > 5000))
    # Unambiguous fire pixels
    test2 = (((ds[sensor_ignorance[sensor]['1.560-1.660']] > 8000) *
              (ds[sensor_ignorance[sensor]['0.433-0.453']] < 2000)) *
             ((ds[sensor_ignorance[sensor]['0.845-0.885']] > 4000) +
              (ds[sensor_ignorance[sensor]['2.100-2.300']] < 1000)).clip(min=0, max=1))
    # other candidate fire pixels
    test3 = (((ds[sensor_ignorance[sensor]['2.100-2.300']]/ds[sensor_ignorance[sensor]['0.845-0.885']]) > 1.8)*
             (ds[sensor_ignorance[sensor]['2.100-2.300']]-ds[sensor_ignorance[sensor]['0.845-0.885']]  > 1700))
    unambiguous = (test1 + test2 + test3).clip(min=0, max=1)
    return(unambiguous)

In [13]:
def get_context_kernel_array(y, x, array):
    
    T, Y, X = array.shape

    ymin = y - 60
    ymax = y + 60
    xmin = x - 60
    xmax = x + 60
    
    if ymin < 0:
        ymin = 0
    
    if xmin < 0:
        xmin = 0

    if ymax > Y:
        ymax = Y
        
    if xmax > X:
        xmax = X

    try:
        outarray = array[0][:, ymin:ymax][xmin:xmax]
    except:
        outarray = np.nans((61,61), dtype=np.float64)
    
    return(outarray, (ymin, ymax, xmin, xmax))

In [14]:
def run_test6(ds):
    #6. ratio b7 b6 > 1.6
    return((ds[sensor_ignorance[sensor]['2.100-2.300']]/ds[sensor_ignorance[sensor]['1.560-1.660']]) > 1.6 )         
    

In [15]:
# Oceans test
#7. {b4 > b5 AND b5 > b6 AND b6 > b7 AND b1 - b7 < 0.2}
def run_test7(ds):
    test7 = ((ds[sensor_ignorance[sensor]['0.630-0.680']]>ds[sensor_ignorance[sensor]['0.845-0.885']])*
             (ds[sensor_ignorance[sensor]['0.845-0.885']]>ds[sensor_ignorance[sensor]['1.560-1.660']])*
             (ds[sensor_ignorance[sensor]['1.560-1.660']]>ds[sensor_ignorance[sensor]['2.100-2.300']])*
             ((ds[sensor_ignorance[sensor]['0.433-0.453']]-ds[sensor_ignorance[sensor]['2.100-2.300']]) < 2000))

    return(test7.clip(min=0, max=1))#.plot()


In [16]:
# Water bodies test - comment - seems like  bad test / smoke complications?
#AND
#8. {(b3 > b2)
def run_test8(ds):    
    test8 = (ds[sensor_ignorance[sensor]['0.525-0.600']]>ds[sensor_ignorance[sensor]['0.450-0.515']])

    return(test8.clip(min=0, max=1))

In [17]:
#OR
#9. (b1 > b2 AND b2 > b3 AND b3 < b4)}.

def run_test9(ds):
    test9 = ((ds[sensor_ignorance[sensor]['0.433-0.453']]>ds[sensor_ignorance[sensor]['0.450-0.515']]) *
            (ds[sensor_ignorance[sensor]['0.450-0.515']]>ds[sensor_ignorance[sensor]['0.525-0.600']])*
            (ds[sensor_ignorance[sensor]['0.525-0.600']]<ds[sensor_ignorance[sensor]['0.630-0.680']]))
  
    return(test9.clip(min=0, max=1))


In [18]:
def get_watermasks(ds):
    watermask=(run_test7(ds)+run_test8(ds)+run_test9(ds)).clip(min=0, max=1)
    return(watermask)

In [19]:
def get_hotspots(ds):
    # Find the candidates and perform context check
    # TODO create mask that is ref7 > 0, non water and not other candidates
    # mask
    # b7=<0
    # water
    # othercandidates
    #candidates = (test1 + test2 + test3).clip(min=0, max=1)
    candidates = get_candidates(ds)
    watermasks = get_watermasks(ds)
    indices = np.where(candidates.data == 1)
    swircandidates = (ds[sensor_ignorance[sensor]['2.100-2.300']].where(candidates.data == 0)).where(watermasks.data == 0)
    nircandidates = (ds[sensor_ignorance[sensor]['0.845-0.885']].where(candidates.data == 0)).where(watermasks.data == 0)

    test4 = (candidates*0)
    test5 = (candidates*0)

    index = 0

    while index < len(indices[1]):
        y = indices[1][index]
        x = indices[2][index]

        #4. ratio between b7 b5 > ratio b7 b5 + max[3x std ratio b7 and b5, 0.8 ]
        #AND
        #5. b7 > b7 + max[3x std b7, 0.08]
        #AND

        #swirkernel = get_context_kernel_array(y,x,ds[sensor_ignorance[sensor]['2.100-2.300']].data)[0]
        #nirkernel = get_context_kernel_array(y,x,ds[sensor_ignorance[sensor]['0.845-0.885']].data)[0]
        swirkernel = get_context_kernel_array(y,x,swircandidates.data)[0]
        nirkernel = get_context_kernel_array(y,x,nircandidates.data)[0] 

        swir = ds[sensor_ignorance[sensor]['2.100-2.300']].data[0][y][x]
        nir = ds[sensor_ignorance[sensor]['0.845-0.885']].data[0][y][x]

        test4.data[0][y][x] = ((swir/nir) > (np.nanmean(swirkernel/nirkernel) + max(3*np.nanstd(swirkernel/nirkernel), 0.8))) 
        test5.data[0][y][x] = (swir > (np.nanmean(swirkernel) + max(3*np.nanstd(swirkernel), 0.08)))

        #print(test4.data[0][y][x],(swir/nir), (np.nanmean(swirkernel/nirkernel) + max(3*np.nanstd(swirkernel/nirkernel), 0.8)))
        #print(test5.data[0][y][x], swir,(np.nanmean(swirkernel) + max(3*np.nanstd(swirkernel), 0.08)) )
        # Write values to new dimension
        #print(index, y, x, get_context_kernel_array(y,x,ds[sensor_ignorance[sensor]['2.100-2.300']].data)[1])
        index = index + 1
    test6 = run_test6(ds)
    t, y, z = np.where((candidates*(test4*test5*test6)).data == 1)
    hotspots = len(y)
    return(hotspots, (candidates*(test4*test5*test6)))
#(candidates*(test4*test5*test6)).plot()

In [20]:
def get_nbr(ds):
    swir = ds[sensor_ignorance[sensor]['2.100-2.300']]
    nir = ds[sensor_ignorance[sensor]['0.845-0.885']]
    return((nir - swir) / (swir + nir))

In [21]:
def get_rdnbr(pre_fire_image, post_fire_imag):
    # Revitalising dNBR from NSW Govt 
    postfire_nbr = get_nbr(post_fire_image)
    prefire_nbr = get_nbr(pre_fire_image)
    dnbr = (prefire_nbr[0] - postfire_nbr[0])
    # Scaling and offset as per NSW Govt algorithm
    #rdnbr = ((dnbr/(np.sqrt(np.abs(prefire_nbr[0]))))*1000)
    return((dnbr/(np.sqrt(np.abs(prefire_nbr[0]))))*1000)

In [22]:
def plot_rgb(image, fake_saturation):
    #image = mask_invalid_data(image)
    rgb = image.to_array(dim='color')
    rgb = rgb.transpose(*(rgb.dims[1:]+rgb.dims[:1]))  # make 'color' the last dimension
    #rgb = rgb.where((rgb <= fake_saturation).all(dim='color'))  # mask out pixels where any band is 'saturated'
    rgb /= fake_saturation  # scale to [0, 1] range for imshow
    rgb[0].plot.imshow(col_wrap=5, add_colorbar=False)
    

In [23]:
def get_timedelta_from_ds(ds, target_time):
    timedelta = {}
    index = 0
    for i in list(ds.time.data):
        timedelta[index] = {"time": i, "delta": target_time - i}
        index = index+1
    #min(list(ds.time.data),key=lambda date : np.datetime64('2019-12-15'))
    return(pd.DataFrame.from_dict(timedelta).transpose())

In [24]:
def get_portion_imaged(image):
    t, y, x = np.where(image[sensor_ignorance[sensor]['2.100-2.300']] >= 0)
    t1, y1, x1 = np.where(image[sensor_ignorance[sensor]['2.100-2.300']] < 0)
    return(len(y)/(len(y1)+len(y)), len(y))

In [25]:
def get_portion_above_threshold(image, threshold):
    y1, x1 = np.where(image)
    y, x = np.where(image > threshold)
    return(len(y)/(len(y1)+len(y)), len(y))

In [96]:
def run_hotspots(output_path, hotspot_index, xtuple, ytuple, hotspot_time_tuple, hotspot_utc_time):

    query = {
        'x': xtuple, 
        'y': ytuple,
        'time': (hotspot_time_tuple),
        'measurements': measurements,
        'output_crs': 'EPSG:3577',
        'resolution': (-30, 30),
        'group_by': 'solar_day'
    }

    try:
        #ds = load_ard(dc=dc, products=sensors_products[sensor], min_gooddata=0.5, **query)

        dataset_list = []
        for product in sensors_products[sensor]:
            datasets = dc.find_datasets(product=product, **query)
            dataset_list.extend(datasets)  

        #ds = load_ard(dc=dc, products=sensors_products[sensor], min_gooddata=0.5, **query)

        ds = dc.load(datasets=dataset_list, **query)

        index = 0

        pd_timediff = get_timedelta_from_ds(ds, hotspot_utc_time) 

        # Initialise variables
        pre_fire_candidate_delta = 0
        post_fire_candidate_delta = 0

        while index < len(ds.time):
            
            image = ds.isel(time=[index])[([sensor_ignorance[sensor]['2.100-2.300'],sensor_ignorance[sensor]['0.845-0.885'],sensor_ignorance[sensor]['0.450-0.515']])]#,'fmask'] )]
            # Maybe we won't use the mask as it confuses smoke with cloud

            mask = ds.isel(time=[index])[('fmask')]

            # Add dataset time to filename
            portion, valid_count = get_portion_imaged(image)

            if (portion > 0.50):

                # Determine pre and post fire imagery closest to hotspot utc time

                if index in pd_timediff[(pd_timediff.delta == pd_timediff.delta.abs())].index:

                    if pre_fire_candidate_delta == 0:
                        pre_fire_candidate_delta = pd_timediff.delta.abs()
                        pre_fire_image = image
                        pre_index = index
                    else:
                        if pre_fire_candidate_delta > pd_timediff.delta.abs():
                            pre_fire_candidate_delta = pd_timediff.delta.abs()
                            pre_fire_image = image
                            pre_index = index
                            
                if index in pd_timediff[(pd_timediff.delta <  pd_timediff.delta.abs())].index:

                    if post_fire_candidate_delta == 0:
                        post_fire_candidate_delta = pd_timediff.delta.abs()
                        post_fire_image = image
                        post_index = index
                    else:
                        if post_fire_candidate_delta > pd_timediff.delta.abs():
                            post_fire_candidate_delta = pd_timediff.delta.abs()
                            post_fire_image = image
                            post_index = index
                # Write rgb to file
                
                #image.isel(time=0).rio.to_raster(output_path.joinpath(str(hotspot_index)+'_'+str(image.time[0].data)+'rgb.tif'))

                #hotspots, hotspot_array = get_hotspots( ds.isel(time=[index]))
                # Write hotspot raster to file

                #hotspot_array.astype('int8').rio.to_raster(output_path.joinpath(str(hotspot_index)+'_'+str(image.time[0].data)+'hotspots.tif'))

                # Add results to hotspot geopandas dataframe
                #print(index, "hotspot pixel count: ", hotspots, " of ", valid_count)

            index = index + 1

    except:
        result = '-'

    # Assuming you have valid data either side of your hotspot date
    try: 
        # Write rgb to file

        pre_fire_image.isel(time=0).rio.to_raster(output_path.joinpath(str(hotspot_index)+'_'+str(pre_fire_image.time[0].data)+'rgb.tif'))
        pre_hotspots, hotspot_array = get_hotspots(ds.isel(time=[pre_index]))
        portion, pre_valid_count = get_portion_imaged(pre_fire_image)
        
        hotspot_array.astype('int8').rio.to_raster(output_path.joinpath(str(hotspot_index)+'_'+str(pre_fire_image.time[0].data)+'hotspots.tif'))
        
    except Exception:
        pass
    try:
        
        post_fire_image.isel(time=0).rio.to_raster(output_path.joinpath(str(hotspot_index)+'_'+str(post_fire_image.time[0].data)+'rgb.tif'))
        post_hotspots, hotspot_array = get_hotspots(ds.isel(time=[post_index]))
        portion, post_valid_count = get_portion_imaged(post_fire_image)
        
        hotspot_array.astype('int8').rio.to_raster(output_path.joinpath(str(hotspot_index)+'_'+str(post_fire_image.time[0].data)+'hotspots.tif'))
        
    except Exception:
        pass
    try:
    
        return(pre_fire_image, post_fire_image, (pre_hotspots/pre_valid_count), (post_hotspots/post_valid_count) )

    except: 
        return()
 

In [97]:
sensors_products = {'oli': ['ga_ls8c_ard_3'], 'msi': ['ga_s2a_ard_nbar_granule', 'ga_s2b_ard_nbar_granule']}
sensor = 'msi'
output_path = Path('s2msi')
output_path.mkdir(parents=True, exist_ok=True)
measurements = []

for measurement in sensor_ignorance[sensor]:
    measurements.append(sensor_ignorance[sensor][measurement])

In [ ]:
# Get time lon and lat for hotspot
for hotspot_index in nearest_points.index:
    #if hotspot_index == 0:
    hotspot_lat = (nearest_points[(nearest_points.index == hotspot_index)].latitude.values[0])
    hotspot_lon = (nearest_points[(nearest_points.index == hotspot_index)].longitude.values[0])
    xtuple, ytuple = buffer_hotspot(hotspot_lon, hotspot_lat)
    hotspot_utc_time = (nearest_points[(nearest_points.index == hotspot_index)].datetime.values[0])  
    hotspot_time_tuple = buffer_date(hotspot_utc_time, 8)
    # Dumb workaround for inexplicable null arrays when trying to execute get_rdnbr in function
    try:

        pre_fire_image, post_fire_image, pre_percent, post_percent = run_hotspots(output_path,hotspot_index, xtuple, ytuple, hotspot_time_tuple, hotspot_utc_time)

        rdnbr = get_rdnbr(pre_fire_image, post_fire_image)

        rdnbr.astype('int16').rio.to_raster(output_path.joinpath(str(hotspot_index)+'_RdNBR.tif'))

        portion, valid_count = get_portion_above_threshold(rdnbr, 1200)

        if (sensor == 'msi'):  
            nearest_points.loc[hotspot_index, "s2msi_rdnbr_gt_1200"] = portion
            nearest_points.loc[hotspot_index, "s2msi_pre_burn_time"] = pre_fire_image.time[0].data
            nearest_points.loc[hotspot_index, "s2msi_post_burn_time"] = post_fire_image.time[0].data
            nearest_points.loc[hotspot_index, "s2msi_pre_burn_timedelta"] = get_timedelta_from_ds(pre_fire_image, hotspot_utc_time).delta[0]
            nearest_points.loc[hotspot_index, "s2msi_post_burn_timedelta"] = get_timedelta_from_ds(post_fire_image, hotspot_utc_time).delta[0]
            nearest_points.loc[hotspot_index, "s2msi_pre_percent"] = pre_percent
            nearest_points.loc[hotspot_index, "s2msi_post_percent"] = post_percent
        if (sensor == 'oli'):
            nearest_points.loc[hotspot_index, "lsoli_rdnbr_gt_1200"] = portion
            nearest_points.loc[hotspot_index, "lsoli_rdnbr_gt_1200"] = portion
            nearest_points.loc[hotspot_index, "lsoli_pre_burn_time"] = pre_fire_image.time[0].data
            nearest_points.loc[hotspot_index, "lsoli_post_burn_time"] = post_fire_image.time[0].data
            nearest_points.loc[hotspot_index, "lsoli_pre_burn_timedelta"] = get_timedelta_from_ds(pre_fire_image, hotspot_utc_time).delta[0]
            nearest_points.loc[hotspot_index, "lsoli_post_burn_timedelta"] = get_timedelta_from_ds(post_fire_image, hotspot_utc_time).delta[0]        
            nearest_points.loc[hotspot_index, "lsoli_pre_percent"] = pre_percent
            nearest_points.loc[hotspot_index, "lsoli_post_percent"] = post_percent
    except:
        run_hotspots(output_path, hotspot_index, xtuple, ytuple, hotspot_time_tuple, hotspot_utc_time)


In [ ]:
pd.pivot_table(nearest_points[(nearest_points['dist_m'] < 5000)],values='count', index=['2_satellite_sensor_product'], columns=['satellite_sensor_product'], aggfunc={'count':len})

In [122]:
# TODO
# likely fire =
# hotspot either side of burn date
# burn scar detected by comparing pre and post burn date
within 24 hours
    print(i)

3 days 23:25:08.976000
3 days 23:25:08.976000
3 days 23:25:08.976000
3 days 23:25:07.976000
3 days 23:25:08.976000



3 days 23:25:08.976000
3 days 23:25:09.976000
3 days 23:24:56.976000
3 days 23:25:08.976000
3 days 23:24:56.976000
3 days 23:25:08.976000
3 days 23:25:09.976000
3 days 23:25:09.976000
3 days 23:25:08.976000
3 days 23:25:09.976000
3 days 23:24:56.976000
3 days 23:25:08.976000
3 days 23:25:08.976000
3 days 23:25:09.976000
3 days 23:25:09.976000
3 days 23:24:56.976000
3 days 23:25:09.976000
3 days 23:25:09.976000
3 days 23:24:56.976000
3 days 23:25:09.976000
3 days 23:25:09.976000
3 days 23:24:56.976000
3 days 23:24:56.976000
3 days 23:25:08.976000
3 days 23:25:09.976000
3 days 23:25:08.976000
3 days 23:25:08.976000
3 days 23:24:55.976000
3 days 23:24:56.976000
3 days 23:25:08.976000
3 days 23:25:08.976000
3 days 23:25:08.976000
3 days 23:25:09.976000
3 days 23:25:08.976000
3 days 23:25:08.976000
3 days 23:25:08.976000
3 days 23:24:55.976000
3 days 23:24:56.976000
3 days 2